## 데이터 및 라이브러리 로드

In [ ]:
import os
import random
import pickle
import numpy as np
import pandas as pd
import gc
import tensorflow as tf
from custom_metric import *
import tensorflow_addons as tfa

SEED = 30
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
%%time
# 약 20초 걸림
with open('D:/inputs/24/train50.pickle', 'rb') as f:
    train = pickle.load(f)
train = train[:, :, :, :10]

with open('D:/inputs/24/train_y50.pickle', 'rb') as f:
    train_y = pickle.load(f)
    train_y = train_y.reshape(train_y.shape[0], 40, 40, 1)

with open('D:/inputs/24/test.pickle', 'rb') as f:
    TEST = pickle.load(f)
TEST = TEST[:, :, :, :10]    


with open('inputs/val_index.pickle', 'rb') as f:
    val_index = pickle.load(f)

VAL_X = train[val_index].copy()
VAL_Y = train_y[val_index].copy()


# 아웃라이어 수정
# train[1,16, 24, -2] = 163.05731201

In [ ]:
with open('inputs/train_50up.pickle', 'rb') as f:
    up50 = pickle.load(f)
up50 = up50[:, :, :, :10]
with open('inputs/train_y_50up.pickle', 'rb') as f:
    up50_y = pickle.load(f)
    up50_y = up50_y.reshape(up50_y.shape[0], 40, 40, 1)

In [ ]:
with open('inputs/train_100up.pickle', 'rb') as f:
    up100 = pickle.load(f)
up100 = up100[:, :, :, :10]
with open('inputs/train_y_100up.pickle', 'rb') as f:
    up100_y = pickle.load(f)
    up100_y = up100_y.reshape(up100_y.shape[0], 40, 40, 1)

In [ ]:
with open('inputs/train_150up.pickle', 'rb') as f:
    up150 = pickle.load(f)
up150 = up150[:, :, :, :10]
with open('inputs/train_y_150up.pickle', 'rb') as f:
    up150_y = pickle.load(f)
    up150_y = up150_y.reshape(up150_y.shape[0], 40, 40, 1)

In [ ]:
with open('inputs/train_200up.pickle', 'rb') as f:
    up200 = pickle.load(f)
up200 = up200[:, :, :, :10]
with open('inputs/train_y_200up.pickle', 'rb') as f:
    up200_y = pickle.load(f)
    up200_y = up200_y.reshape(up200_y.shape[0], 40, 40, 1)

In [ ]:
print(up50.shape)
print(up100.shape)
print(up150.shape)
print(up200.shape)

train = np.vstack([train, up50])

train = np.vstack([train, up100])

# train = np.vstack([train, up150])


# train = np.vstack([train, up200])


del up50, up100, up150, up200

In [ ]:
train_y = np.vstack([train_y, up50_y])

train_y = np.vstack([train_y, up100_y])

# train_y = np.vstack([train_y, up150_y])

# train_y = np.vstack([train_y, up200_y])


del up50_y, up100_y, up150_y, up200_y

In [ ]:
train.shape

In [ ]:
gc.collect()

## target 값 log 변환

In [ ]:
train_y = np.log(train_y+1)
VAL_Y = np.log(VAL_Y+1)

## 회전변환

In [ ]:
## 회전변환시 마이너스(-) 하는 경우, 제곱을 해주면 target값과의 상관관계가 높아짐. 테스트해볼것!
# v1_m_h1 = ((train[:, :, :, 0] * np.cos(np.pi / 4) - train[:, :, :, 1] * np.sin(np.pi / 4)) ** 0.5) / 100

In [ ]:
from preprocessing import rotation

In [ ]:
train = rotation(train)
TEST = rotation(TEST)
VAL_X = rotation(VAL_X)

## one-hot encoding

In [ ]:
def one_hot_encoding(data):
    data9 = data[:, :, :, 9].copy()
    data9 = data9.reshape(data9.shape[0], data9.shape[1], data9.shape[2], 1)
    data = np.concatenate([data, data9.copy()], -1)
    data = np.concatenate([data, data9.copy()], -1)
    data = np.concatenate([data, data9.copy()], -1)
    data[:,:,:,10] = data[:,:,:,10] // 100
    data[:,:,:,11] = data[:,:,:,11] // 100
    data[:,:,:,12] = data[:,:,:,12] // 100
    data[:,:,:,10] = np.where(data[:,:,:,10] != 1, 0, 1)
    data[:,:,:,11] = np.where(data[:,:,:,11] != 2, 0, 1)
    data[:,:,:,12] = np.where(data[:,:,:,12] != 3, 0, 1)
    return data

In [ ]:
train = one_hot_encoding(train)
TEST = one_hot_encoding(TEST)
VAL_X = one_hot_encoding(VAL_X)

In [ ]:
train.shape # 01, 23, 4, 56, 78, 9, 10, 11, 12

In [ ]:
TEST.shape

In [ ]:
VAL_X.shape

In [ ]:
train[:, :, :, 9] = train[:, :, :, 9] / 322
TEST[:, :, :, 9] = TEST[:, :, :, 9] / 322
VAL_X[:, :, :, 9] = VAL_X[:, :, :, 9] / 322

In [ ]:
# 4개의 경도, 위도 정보 삭제함.
# train = train[:, :, :, :10]

In [ ]:
# 4개의 경도, 위도 정보 삭제함.
# TEST = TEST[:, :, :, :10]

## augmentation

In [ ]:
train = np.concatenate([train, train_y], -1)

In [ ]:
%%time

train1 = np.rot90(train, 1, (1,2))
train2 = np.rot90(train, 2, (1,2))
train3 = np.rot90(train, 3, (1,2))
train_lr = np.fliplr(train)
# train_ud = np.flipud(train)

train = np.vstack([train, train1])
del train1

train = np.vstack([train, train2])
del train2

train = np.vstack([train, train3])
del train3

train = np.vstack([train, train_lr])
del train_lr

# train = np.vstack([train, train_ud])
# del train_ud

In [ ]:
train_y = train[:, :, :, -1].copy()
train_y = train_y.reshape(train_y.shape[0], train_y.shape[1], train_y.shape[2], 1)
train = train[:,:,:,:-1]

In [ ]:
gc.collect()

## all_np로 결합 후 scaling

In [ ]:
## ss 약 5분 걸림
# from preprocessing import scaling

# train, test = scaling(train, test, 'ss') # 3번째 인자 'rs', 'ms', 'ss' 


## 데이터셋 나누기

In [ ]:
from sklearn.model_selection import train_test_split
train, test, train_y, test_y = train_test_split(train, train_y, test_size=0.025, random_state=SEED)

## 모델 선언 - Unet, Resnet

In [ ]:
from models import build_model, build_model2
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D, Add, BatchNormalization, concatenate, add, MaxPooling2D, Conv2DTranspose, Activation
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

In [ ]:
input_layer = Input((40, 40, train.shape[3]))
output_layer = build_model(input_layer, 32)

# sgd = tf.keras.optimizers.SGD(0.01)
# stocastic_avg_opt = tfa.optimizers.SWA(sgd)

# ad = tf.keras.optimizers.Adam(0.01)
# stocastic_avg_opt = tfa.optimizers.SWA(ad)
# model.compile(loss="mae", optimizer=stocastic_avg_opt, metrics=["mae"])
#


# model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])

scores = []

In [ ]:
input_layer = Input(train.shape[1:])
output_layer = build_model(input_layer, 32)
model = Model(input_layer, output_layer)

In [ ]:
def deep_cnn_advanced_nin():
    model = Sequential()
    model.add(Conv2D(input_shape = (train.shape[1:]), filters = 50, kernel_size = (3,3), strides = (1,1), padding = 'same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 50, kernel_size = (3,3), strides = (1,1), padding = 'same', kernel_initializer='he_normal'))
    # 1x1 convolution
    model.add(Conv2D(filters = 25, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Conv2D(filters = 50, kernel_size = (3,3), strides = (1,1), padding = 'same', kernel_initializer='he_normal'))
    # 1x1 convolution
    model.add(Conv2D(filters = 25, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 50, kernel_size = (3,3), strides = (1,1), padding = 'same', kernel_initializer='he_normal'))
    # 1x1 convolution
    model.add(Conv2D(filters = 25, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Conv2D(filters = 50, kernel_size = (3,3), strides = (1,1), padding = 'same', kernel_initializer='he_normal'))
    # 1x1 convolution
    model.add(Conv2D(filters = 25, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 50, kernel_size = (3,3), strides = (1,1), padding = 'same', kernel_initializer='he_normal'))
    # 1x1 convolution
    model.add(Conv2D(filters = 25, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(25, (3, 3), strides=(2, 2), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 25, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(25, (3, 3), strides=(2, 2), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(1, (1,1), padding="same", activation='relu'))

    return model

In [ ]:
def auto_encoder():
    inputs=Input(train.shape[1:])
    conv1 = Conv2D(30, (1, 1), activation="relu", padding="same")(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(20, (1, 1), activation="relu", padding="same")(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(30, (1, 1), activation="relu", padding="same")(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2DTranspose(40, (1, 1), padding="same")(conv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(conv1)
    model=Model(inputs=inputs, outputs=output_layer)
    return model

In [ ]:
def resnet_model2(start_neurons):

    inputs=Input((40,40,13))
    conv1 = Conv2D(start_neurons * 1, (2, 2), activation="relu", padding="same")(inputs)
    conv1 = BatchNormalization()(conv1)
    
    pool1 = MaxPooling2D((2, 2))(conv1)
    
    conv2 = Conv2D(start_neurons * 1, (1, 1), activation="relu", padding="same")(pool1)
    conv2 = BatchNormalization()(conv2)

    conv2 = Conv2D(start_neurons * 1, (2, 2), activation="relu", padding="same")(conv2)
    conv2 = BatchNormalization()(conv2)

    conv2 = Conv2D(start_neurons * 4, (1, 1), activation="relu", padding="same")(conv2)
    conv2 = BatchNormalization()(conv2)


    pool1 = Conv2D(start_neurons * 4, (1, 1), activation="relu", padding="same")(pool1)
    pool1 = BatchNormalization()(pool1)

    add1 = add([conv2, pool1])


    conv3 = Conv2D(start_neurons * 1, (1, 1), activation="relu", padding="same")(add1)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(start_neurons * 1, (2, 2), activation="relu", padding="same")(conv3)
    conv3 = BatchNormalization()(conv3)

    conv3 = Conv2D(start_neurons * 4, (1, 1), activation="relu", padding="same")(conv3)
    conv3 = BatchNormalization()(conv3)
    add2 = add([conv3, add1])
    
    conv4 = Conv2D(start_neurons * 1, (1, 1), activation="relu", padding="same")(add2)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(start_neurons * 1, (2, 2), activation="relu", padding="same")(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(start_neurons * 4, (1, 1), activation="relu", padding="same")(conv4)
    conv4 = BatchNormalization()(conv4)

    add3 = add([conv4, add2])


    conv5 = Conv2D(start_neurons * 2, (1, 1), activation="relu", padding="same")(add3)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(start_neurons * 2, (2, 2), activation="relu", padding="same")(conv5)
    conv5 = BatchNormalization()(conv5)

    conv5 = Conv2D(start_neurons * 8, (1, 1), activation="relu", padding="same")(conv5)
    conv5 = BatchNormalization()(conv5)
    
    conv6 = Conv2D(start_neurons * 8, (1, 1), activation="relu", padding="same")(add3)
    conv6 = BatchNormalization()(conv6)

    add4 = add([conv5, conv6])
    
    deconv = Conv2DTranspose(start_neurons * 2, (1, 1), padding="same")(add4)
    conv7 = Conv2D(start_neurons * 4, (1, 1), activation="relu", padding="same")(deconv)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(start_neurons * 2, (1, 1), activation="relu", padding="same")(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(start_neurons * 1, (1, 1), activation="relu", padding="same")(conv7)
    conv7 = BatchNormalization()(conv7)
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(conv7)
    model=Model(inputs=inputs, outputs=output_layer)
    return model

In [ ]:
# kernel_initializer='he_normal'
def resnet_model():
    inputs=Input(train.shape[1:])
    
    bn=BatchNormalization()(inputs)
    conv0=Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(bn)
    
    bn=BatchNormalization()(conv0)
    conv=Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(bn)
    concat=concatenate([conv0, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    conv=Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(bn)
    concat=concatenate([concat, conv], axis=3)
    
    # 원래는 5였음 - 6분정도 걸리고 2로 줄이면 3분정도 걸림 /에폭당
    for i in range(3):
        bn=BatchNormalization()(concat)
        conv=Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(bn)
        concat=concatenate([concat, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    outputs=Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')(bn)
    
    model=Model(inputs=inputs, outputs=outputs)
    
    return model

In [ ]:
model = tf.keras.models.load_model('epochs_8.h5')
model.summary()

In [ ]:
indexes = []
for train_idx in k_fold.split(train):
    indexes.append(train_idx)

In [ ]:
# 인덱스 로드해서 잘 복원되는지 확인해볼것.
# import pickle
# with open('submissions/22_23_kfold_index.pickle', 'wb') as f:
#     pickle.dump(indexes, f)

In [ ]:
indexes[3]

## k-fold 모델 훈련 및 모델 저장

In [ ]:
from sklearn.model_selection import KFold

FOLD = 4
k_fold = KFold(n_splits=FOLD, shuffle=True, random_state=SEED)

EPOCHS = 100
history = []
scores = []

result = 0

In [ ]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)

In [ ]:
model_number = 0


for train_idx, val_idx in k_fold.split(train, train_y):
    x_train, y_train = train[train_idx], train_y[train_idx]
    x_val, y_val = train[val_idx], train_y[val_idx]
    print(x_train.shape)

#     U net
#     model = Model(input_layer, output_layer)
    
#     resnet 모델 - 지금까지는 베스트
    model = resnet_model()
    
#     resnet 모델2
#     model = resnet_model2(13)
    
#     auto-encoder 모델
#     model = auto_encoder()
    
#   1x1 convolution 모델
#     model = deep_cnn_advanced_nin()
    
    model.compile(loss="mae", optimizer="adam", metrics=["mae"])
    
    es = EarlyStopping(patience=5, verbose=1)
    mc = ModelCheckpoint(f'best_{model_number}.h5', save_best_only=True, verbose=1)
    rlp = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.8)
    csv_logger = CSVLogger(f'training_{model_number}.csv')
    
    model_ = model.fit(x_train, y_train, epochs = EPOCHS, validation_data=(VAL_X, VAL_Y), verbose=1, batch_size = 64, callbacks = [es, mc, rlp, csv_logger])
    
    history.append(model_)
    model.load_weights(f'best_{model_number}.h5')
    result += model.predict(TEST) / FOLD
    
    pred = model.predict(test)
    score = maeOverFscore(test_y, np.exp(pred)-1)
    print('mae / fscore: ' + str(score) + '\n')
    scores.append(score)
    print(scores)
    pred = model.predict(VAL_X)
    score = maeOverFscore(VAL_Y, np.exp(pred)-1)
    print('VAL_X  mae / fscore: ' + str(score) + '\n')
    model_number += 1

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,5))
plt.plot(history[0].history['val_loss'])

## 리더보드와 비슷한 테스트셋 찾는 중

In [ ]:
import random

model = resnet_model()
model.compile(loss="mae", optimizer="adam", metrics=["mae"])
sample_score = 0

while sample_score < 1.48:
    sample = random.sample(range(30662),2416)
    sample_x = train[sample]
    sample_y = train_y[sample]
    
    tmp = 0
    
    for i in range(2):
        model.load_weights(f'best_{i}.h5')
        res = model.predict(sample_x)
        tmp += (np.exp(res)-1) / 2
        
    sample_score = maeOverFscore(sample_y, tmp)
    print(sample_score)
    gc.collect()

In [ ]:
import pickle
with open('inputs/val_index.pickle', 'wb') as f:
    pickle.dump(sample, f)

In [ ]:
import pickle
with open('inputs/val_x.pickle', 'wb') as f:
    pickle.dump(train[sample], f)


In [ ]:
import pickle
with open('inputs/val_y.pickle', 'wb') as f:
    pickle.dump(train_y[sample], f)

## 자체 test셋으로 검증
리더보드와 매우 비슷

In [ ]:
# with open('inputs/my_test_set.pickle', 'rb') as f:
#     test, test_y = pickle.load(f)

In [ ]:
model_score = 0
tmp = 0
for i in range(4):
#     model = Model(input_layer, output_layer)
#     model.compile(loss="mae", optimizer="adam", metrics=["mae"])
    model.load_weights(f'best_{i}.h5')
    res = model.predict(VAL_X)
    tmp += (np.exp(res)-1) / 4
    
model_score = maeOverFscore(VAL_Y, tmp)
model_score

In [ ]:
model_score = 0
tmp = 0
for i in range(4):
#     model = Model(input_layer, output_layer)
#     model.compile(loss="mae", optimizer="adam", metrics=["mae"])
    model.load_weights(f'best_{i}.h5')
    res = model.predict(test)
    tmp += (np.exp(res)-1) / 4
    
model_score = maeOverFscore(test_y, tmp)
model_score

## TEST로 submission 생성

In [ ]:
result = 0
for i in range(4):
#     model = Model(input_layer, output_layer)
#     model.compile(loss="mae", optimizer="adam", metrics=["mae"])
    model.load_weights(f'best_{i}.h5')
    res = model.predict(TEST)
    result += (np.exp(res)-1) / 4
    

In [ ]:
submission = pd.read_csv('inputs/sample_submission.csv')
submission.iloc[:,1:] = result.reshape(-1, 1600)
submission.to_csv('kfold_unet.csv', index = False)

In [ ]:
tmp32 = pd.read_csv('submissions/32_kfold_resnet2_log2.csv')
tmp34 = pd.read_csv('submissions/34_kfold_unet.csv')

tmp35 = tmp32.copy()
tmp35.iloc[:, 1:] = 0.8 * tmp32.iloc[:, 1:] + 0.2 * tmp34.iloc[:, 1:]

In [ ]:
tmp35.to_csv('tmp36.csv', index=False)

## 앙상블

In [ ]:
tmp1 = pd.read_csv('22_unet_1.csv')
tmp2 = pd.read_csv('23_kfold_resnet_4.csv')

tmp1.iloc[:, 1:] = (tmp1.iloc[:, 1:] * 0.5) + (tmp2.iloc[:, 1:] * 0.5)

tmp1.to_csv('ensemble_22_23.csv', index = False)

## 해야할 일

1. ~~스케일링(Scaling) - Robust, log, Standard Scaler  ~~  
스케일링 방법을 다르게 해야하나. 하나의 사진을 기준으로 처리해야 할지 아니면 1번픽셀, 2번픽셀... 1600픽셀 끼리 해야하나
2. ~~위도 경도 피쳐를 가지고 위도+경도, 위도-경도 피쳐 만들기(45도 회전변환)~~: 30, 60도 변환 추가
3. 현재 custom metric이 작동하지 않으므로, 이 또한 수정해야함.
4. tree모델 만들고, 앙상블
5. ~~target 값에 log 취하기~~: 점수 안오름
6. 순서가 바뀌긴 했으나, EDA 작업하자.
7. 강수량 타일이 10개 미만인 것의 평균, 20개미만의 평균.... 50개 미만의 평균... 알아보자
8. ~~augumentation 사용해서 강수량이 일정수준 이상인 사진 augementation 하자~~ - 다만, regression인 만큼 가장자리 어떻게 처리할지 생각해보자. - 정사각형 사진이기에 90도 돌린사진, 180도 돌린사진, 270도 돌린사진 이렇게만 해도 데이터가 증식될거같음 -  
**그런데!!!!!** 생각해봐야할것이.....  
위도, 경도 등 여러 요소가 존재할텐데 돌려버리면 원래는 비 안오는거를 비온다고 볼 수도 있을 듯 싶다. 쉽게 예를 들면 I 모양일때는 비가 오고, ㅡ 모양일때는 비가안오는데, 돌리는 바람에 안오는걸 온다고 할 수도 있을 것 같다. - 이건 테스트해볼수밖에 없을 듯 싶다.  
돌릴수있는방법도 여러가지이다. 90도씩회전하기, 수직축기준으로 전환, 수평축기준으로 전환 등 - 캐글에서는 수직축, 수평축으로 전환해서 augmentation 썼다고 함.  위성이 도는 방향을 보면, 270도 돌리는 것도 괜찮을듯
9. pretrained 모델 활용방안 생각하기 - 최소 사진크기가 존재하는데 우리는 40x40이라 작은편.. 이걸 40x40을 이어 붙여서 200x200으로 만들어서 넣으면 될거같은데 시도해보자.
10. 9번채널 카테고리 변수 원핫인코딩
11. Unet 모델과 FPN 모델 앙상블해서 제출하자
12. loss='mse', metric=['mse', 'mae'] 이렇게 해보자
13. U_res_net 구현하기
14. 회전변환한 걸로 U_net 돌려서 제출하기
15. off nadir viewing angle, earth incidence angle

| 리더보드(5부터) | mae / fscore | mae | 비고 |
|---|:---:|---:|---:|
| 1.93737 | 1.8744 | 0.2451 |  |
| 1.75177 | 1.7016 | 0.2384 |  |
| 1.61793 | 1.4234 | 0.0832 | 리더보드랑 metric 점수랑 차이 많이 남. 해결해야함. |